# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 1: 

Να ταξινομημηθούν, σε φθίνουσα σειρά, οι ηλικιακές ομάδες των θυμάτων σε περιστατικά που περιλαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης”. 
Ηλικιακές ομάδες:
- Παιδιά: < 18
- Νεαροί ενήλικοι: 18-24
- Ενήλικοι: 25-64
- Ηλικιωμένοι: > 64

*Σημείωση: Ως εγκλήματα που περιλαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης” θεωρούμε όλα εκείνα τα περιστατικά που περιέχουν τον όρο “aggravated assault” στη σχετική περιγραφή*

In [1]:
from pyspark.sql import SparkSession
import time

spark = SparkSession \
    .builder \
    .appName("LACrimeData") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1033,application_1732639283265_1000,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None

In [4]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1034,application_1732639283265_1001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
983,application_1732639283265_0950,pyspark,idle,Link,Link,None,
984,application_1732639283265_0951,pyspark,idle,Link,Link,None,
986,application_1732639283265_0953,pyspark,idle,Link,Link,None,
993,application_1732639283265_0960,pyspark,idle,Link,Link,None,
996,application_1732639283265_0963,pyspark,idle,Link,Link,None,
999,application_1732639283265_0966,pyspark,idle,Link,Link,None,
1001,application_1732639283265_0968,pyspark,idle,Link,Link,None,
1003,application_1732639283265_0970,pyspark,idle,Link,Link,None,
1004,application_1732639283265_0971,pyspark,idle,Link,Link,None,
1010,application_1732639283265_0977,pyspark,idle,Link,Link,None,


In [6]:
conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 4

### Διάβασμα των CSV αρχείων

In [7]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, TimestampType

# Ορισμός του schema του dataset με βάση την περιγραφή
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", TimestampType()),
    StructField("DATE OCC", TimestampType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Διαβάζουμε τα 2 datasets (2010-2019 και 2020-σήμερα) και τα συνενώνουμε σε 1
crime_data_2010_2019 = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)
crime_data_2020_present = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, schema=crimes_schema)
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Υλοποίηση με DataFrame

In [13]:
from pyspark.sql.functions import col, udf
import time

# Δημιουργούμε μία συνάρτηση για τις ηλικιακές ομάδες
def get_age_group(age):
    try:
        age = int(age)
        if age < 18:
            return "Children"
        if age <= 24:
            return "Young adults"
        if age <= 64:
            return "Adults"

        return "Seniors"

    except Exception as e:
        return "Unknown"

start_time = time.time()

# Περισταστικά που περιλαμβάνουν οποιαδήποτε μορφή "βαριάς σωματικής βλάβης" (aggravated assault)
agg_assaults = crime_data.filter(crime_data["Crm Cd Desc"].contains("AGGRAVATED ASSAULT"))

# Μετατροπή συνάρτησης με UDF (user defined function)
get_age_group_udf = udf(get_age_group, StringType())

# Δημιουργούμε μία επιπλέον στήλη για τις ηλικιακές ομάδες
agg_assaults = agg_assaults.withColumn("Age Group", get_age_group_udf(col("Vict Age")))

# Μετράμε (count) τα περιστατικά βαριάς σωματικής βλάβης ανά ηλικιακή ομάδα,
# και τα ταξινομούμε σε φθίνουσα σειρά
result_df = agg_assaults.groupBy("Age Group") \
            .count() \
            .orderBy("count", ascending=False)

result_df.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|   Age Group| count|
+------------+------+
|      Adults|121093|
|Young adults| 33605|
|    Children| 15928|
|     Seniors|  5985|
+------------+------+

In [14]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 3.01 seconds

### Υλοποίηση με RDD

In [17]:
# Διάβασμα του CSV σε RDD μορφή
sc = spark.sparkContext
# crime_data_2010_2019_rdd = sc.textFile(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
#                             .map(lambda x: (x.split(",")))
# crime_data_2020_present_rdd = sc.textFile(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
#                             .map(lambda x: (x.split(",")))
# crime_data_rdd = crime_data_2010_2019_rdd.union(crime_data_2020_present_rdd)

# # Αφαίρεση της κεφαλίδας του CSV
# csv_header = crime_data_rdd.first()
# crime_data_rdd = crime_data_rdd.filter(lambda row: row != csv_header)

crime_data_rdd = crime_data.rdd

start_time = time.time()

# Περισταστικά που περιλαμβάνουν οποιαδήποτε μορφή "βαριάς σωματικής βλάβης" (aggravated assault)
agg_assaults_rdd = crime_data_rdd.filter(lambda row: "AGGRAVATED ASSAULT" in row[9])

# Δημιουργούμε έναν νέο πίνακα με 2 key: ηλικιακή ομάδα, value: 1
agg_assaults_rdd = agg_assaults_rdd.map(lambda row: (get_age_group(row[11]), 1))

# Μετράμε (reduce) τα περιστατικά βαριάς σωματικής βλάβης ανά ηλικιακή ομάδα (key), και τα ταξινομούμε σε φθίνουσα σειρά
result_rdd = agg_assaults_rdd \
                .reduceByKey(lambda a, b: a + b) \
                .sortBy(lambda x: x[1], ascending=False)

result_rdd = result_rdd.collect()

print(result_rdd)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Adults', 121093), ('Young adults', 33605), ('Children', 15928), ('Seniors', 5985)]

In [18]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 17.96 seconds

## Συμπεράσματα

Η υλοποίηση με χρήση του **DataFrame API** ήταν αρκετά γρηγορότερη από αυτήν με το **RDD API**. 

Συγκεκριμένα, η υλοποίηση του ερωτήματος (χωρίς την ανάγνωση των CSV αρχείων) χρειάστηκε 3.01 δευτερόλεπτα με DataFrames και 17.96 δευτερόλεπτα με RDD.

Πράγματι, θα αναμέναμε το query να είναι γρηγορότερο με τη χρήση DataFrames, αφού τα RDDs δε μπορούν να βελτιστοποιηθούν από το **Spark** (*Catalyst Optimizer*).

Επίσης, τα RDDs είναι γενικότερα πιο αργά σε non-JVM γλώσσες, όπως η Python στην οποία εκτελέστηκαν τα ερωτήματα.